In [39]:
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
from tqdm import tqdm
tqdm.pandas()

tickers = ["TSLA", "MSFT", "AAPL", "NVDA"]

In [40]:
data = yf.download(tickers=tickers, start="2020-01-01", end="2025-01-01", group_by="ticker").dropna()

C:\Users\admin\AppData\Local\Temp\ipykernel_19588\1633098113.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=tickers, start="2020-01-01", end="2025-01-01", group_by="ticker").dropna()
[*********************100%***********************]  4 of 4 completed


In [84]:
def generate_signal(row):
    return np.random.choice([1, 0])

def add_signal(df):
    df = df.copy()
    df["Signal"] = df.apply(lambda row: generate_signal(row), axis=1)
    return df

def add_atr(df, length=14):
    df = df.copy()
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=length)
    return df

def preprocess_ticker(df):
    df = df.copy()
    df = add_signal(df)
    df = add_atr(df)

    df = df.dropna(subset=["ATR", "Signal"]).reset_index(drop=True)
    return df

In [85]:
from backtesting import Backtest, Strategy

class RandomSignalsStrategy(Strategy):
    base_size = 0.03
    sl_ratio = 3
    clamp_val = 0.2
    smooth_f = True

    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.atr = self.I(lambda: self.data.ATR)
        self.allow_short = False
        self.pnl_hist = []
        if self.smooth_f:
            self.prev_f = 0
        

    def calculate_f(self):
        if len(self.pnl_hist) < 10:
            return self.base_size
        
        pnl = np.array(self.pnl_hist)

        w = pnl[pnl>0]

        if len(w) == 0:
            return self.base_size
        
        l = pnl[pnl<0]

        p = len(w)/len(pnl)
        q = 1 - p
        b = w.mean() / abs(l.mean())
        f = p - q/b
        
        f = np.clip(f, -self.clamp_val, self.clamp_val)

        if self.smooth_f:
            f = 0.3 * self.prev_f + 0.7 * f
            self.prev_f = f

        return f
        


    def next(self):
        if np.isnan(self.data.ATR[-1]) or np.isnan(self.signal[-1]):
            return
        
        f = self.calculate_f()

        for trade in self.trades:
            ts = self.sl_ratio * self.data.ATR[-1]
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - ts)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + ts)

        curr_close = self.data.Close[-1]

        if self.signal[-1] == 1 and not self.position:
            sl = curr_close - self.sl_ratio * self.data.ATR[-1]
            if not np.isnan(sl):
                self.buy(size=f, sl=sl)

        elif self.signal[-1] == 0 and not self.position and self.allow_short:
            sl = curr_close + self.sl_ratio * self.data.ATR[-1]
            if not np.isnan(sl):
                self.sell(size=f, sl=sl)

    def after_trade(self, trade):
        self.pnl_hist.append(trade.pl)


In [86]:
results = []
dataframes = {}

for ticker in tickers:
    print(f"Running backtest for {ticker}...")
    df = data[ticker].copy()
    df = preprocess_ticker(df)
    dataframes[ticker] = df

    bt = Backtest(df, RandomSignalsStrategy, cash=1000, commission=0.0002, trade_on_close=True)
    stats = bt.run()
    results.append({
        "Ticker": ticker,
        "Return_%": stats["Return [%]"],
        "Sharpe": stats["Sharpe Ratio"],
        "Win Rate": stats["Win Rate [%]"],
        "Trades": stats["# Trades"]
    })

    

Running backtest for TSLA...
Running backtest for MSFT...
Running backtest for AAPL...
Running backtest for NVDA...


C:\Users\admin\AppData\Local\Temp\ipykernel_19588\2783625548.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, RandomSignalsStrategy, cash=1000, commission=0.0002, trade_on_close=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_19588\2783625548.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, RandomSignalsStrategy, cash=1000, commission=0.0002, trade_on_close=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_19588\2783625548.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, RandomSignalsStrategy, cash=1000, commission=0.0002, trade_on_close=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_19588\2783625548.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, RandomSignalsStrategy,

In [87]:
results_df = pd.DataFrame(results)
print("\nPer-asset performance:")
print(results_df.round(3))

agg_return = sum(results_df["Return_%"])
agg_sharpe = results_df["Sharpe"].mean()

print("\nAggregate Portfolio Performance:")
print(f"Aggregate Return [%]: {agg_return:.2f}")
print(f"Average Sharpe Ratio: {agg_sharpe:.2f}")



Per-asset performance:
  Ticker  Return_%  Sharpe  Win Rate  Trades
0   TSLA    11.150     NaN   100.000     1.0
1   MSFT     0.000     NaN       NaN     0.0
2   AAPL     0.000     NaN       NaN     0.0
3   NVDA     8.513     NaN    42.857    28.0

Aggregate Portfolio Performance:
Aggregate Return [%]: 19.66
Average Sharpe Ratio: nan
